In [ ]:
!pip install ../input/pycocotool/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl
!pip install -q "../input/zoohpa/pytorch_zoo-master"
!pip install -q "../input/effpytorch/EfficientNet-PyTorch-master"
!pip install "../input/hpacellsegmentatorraman/HPA-Cell-Segmentation/"

In [ ]:
import torch
import pytorch_lightning as pl
import torchvision.models as models


import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


from skimage import io
from skimage.transform import resize
from skimage.color import rgb2gray,gray2rgb

from matplotlib import pyplot as plt

import csv
import pandas as pd
import numpy as np
import cv2

In [ ]:
#import sys
#import os
#sys.path.insert(1, '../input/cellsegm/HPA-Cell-Segmentation-master')

#import hpacellseg.cellsegmentator as cellsegmentator
#from hpacellseg.utils import label_cell, label_nuclei

from hpacellseg.cellsegmentator import *
from hpacellseg import cellsegmentator, utils

import base64


from pycocotools import _mask as coco_mask
import typing as t
import zlib


NUC_MODEL = '../input/requiredfiles/nuclei-model.pth'
CELL_MODEL = '../input/requiredfiles/cell-model.pth'

# segmentator = cellsegmentator.CellSegmentator(
#     NUC_MODEL,
#     CELL_MODEL,
#     scale_factor=0.25,
#     device="cuda",
#     padding=True,
#     multi_channel_model=True,
# )


segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    device="cuda",
    multi_channel_model=True,
)
def extract_data_from_folder_and_create_csv(path = "../input/hpa-single-cell-image-classification/test/"):
    filelist= [file for file in os.listdir(path) if file.endswith('.png')]
    
    trimmed_list = [None]*(int(len(filelist)/4))
    count = 0
    for i in (filelist):
        trim_str = i.partition('_')[0]
        #print(trim_str)
        if trim_str not in trimmed_list: 
            #trimmed_list.append(trim_str) 
            trimmed_list[count] = trim_str
            count +=1
    return trimmed_list

def extract_data_from_csv(csvpath):
    fields = [] 
    rows = [] 
    filename = csvpath
    with open(filename, 'r') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 

        fields = next(csvreader)
        # extracting each data row one by one 
        for row in csvreader: 
            rows.append(row) 
            
    return rows

# reading csv file
def get_segmented_image_and_label(container,index):
    
    '''
    Takes csv_path and index as an input
    Returns Segmented image and label if there is only single label
    '''

    Data = container[index]
    image_id = Data[0]
    #print(Data)
    img_blue = io.imread("../input/hpa-single-cell-image-classification/test/"+ image_id+"_blue.png")
    img_green = io.imread("../input/hpa-single-cell-image-classification/test/"+ image_id+"_green.png")
    img_red = io.imread("../input/hpa-single-cell-image-classification/test/"+ image_id+"_red.png")
    img_yellow = io.imread("../input/hpa-single-cell-image-classification/test/"+ image_id+"_yellow.png")
    divider = 1

    img_blue_resized = resize(img_blue, (img_blue.shape[0] // divider, img_blue.shape[1] // divider),
                   anti_aliasing=True)
    img_green_resized = resize(img_green, (img_green.shape[0] // divider, img_green.shape[1] // divider),
                   anti_aliasing=True)
    img_red_resized = resize(img_green, (img_red.shape[0] // divider, img_red.shape[1] // divider),
                   anti_aliasing=True)
    img_yellow_resized = resize(img_yellow, (img_yellow.shape[0] // divider, img_yellow.shape[1] // divider),
                   anti_aliasing=True)


    rgb = np.dstack((img_blue_resized,img_green_resized + img_yellow_resized,img_red_resized))

    
    nuc_segmentations = segmentator.pred_nuclei([rgb[:,:,2]])

    # For full cells
    cell_segmentations = segmentator.pred_cells([rgb],precombined = True)


    #for i, pred in enumerate(cell_segmentations):
    nuclei_mask, cell_mask = utils.label_cell(nuc_segmentations[0], cell_segmentations[0])


    return rgb,cell_mask,image_id




def binary_mask_to_rle(mask):
    #print("Before")
    #print(mask)
    mask[mask >= 1] = 1
    mask = mask.astype(np.bool) 
    #print("After")
    #print(mask)
#     if mask.dtype != np.bool:
#         raise ValueError(
#         "encode_binary_mask expects a binary mask, received dtype == %s" %
#         mask.dtype)
    
    
    mask = np.squeeze(mask)
    
#     if len(mask.shape) != 2:
#         raise ValueError(
#         "encode_binary_mask expects a 2d mask, received shape == %s" %
#         mask.shape)

    # convert input mask to expected COCO API input --
    
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    #print(mask_to_encode.shape)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)
    #print(mask_to_encode)
    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str.decode('ascii')


def convert_to_binary(seg_mask):
    cell_mask = seg_mask[..., np.newaxis] # Added a dimension for channel
    _,thresh = cv2.threshold(cell_mask,0,255,cv2.THRESH_BINARY) # binarize
    
    #thresh_norm = cv2.normalize(src=thresh, dst=None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    return thresh

In [ ]:
# Class Architecture
from efficientnet_pytorch import EfficientNet

class EffNet(pl.LightningModule):
    
    def __init__(self,learning_rate=1e-4):
        super().__init__()
        self.effnet = EfficientNet.from_name('efficientnet-b3')
        in_features = self.effnet._fc.in_features
        self.effnet._fc = nn.Linear(in_features,19)
        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()
        self.learning_rate = learning_rate

            
    def forward(self, x):
        raw_predit = self.effnet(x)
        return raw_predit

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=(self.learning_rate))
        #optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        #scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1)
        
        #return [optimizer], [scheduler]
        return optimizer
    
    def loss_function(self,x_hat, x):
        creterion = nn.CrossEntropyLoss()
        loss = creterion(x_hat,x)
        return loss
        
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self(x)
        loss = self.loss_function(y_hat, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True,prog_bar=True,logger=True)
        self.train_acc(y_hat, y)
        self.log('train_acc', self.train_acc, on_step=True, on_epoch=True,prog_bar=True,logger=True)
        #log = {'train_loss' : loss, 'train_acc': self.train_acc}
        #return {'train_log':log}
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_hat = self(x) 
        loss = self.loss_function(y_hat, y)
        self.log('val_loss', loss, on_step=True, on_epoch=True,prog_bar=True,logger=True)
        self.valid_acc(y_hat, y)
        self.log('val_acc', self.valid_acc,on_step=True, on_epoch=True,prog_bar=True,logger=True)
        #log = {'val_loss' : loss, 'val_acc': self.valid_acc}
        #return {'val_log':log}
        return loss
        


In [ ]:
#model = EffNet(3e-4)
#import time

#Testing 

#sub = pd.DataFrame()
sub = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
#container = extract_data_from_folder_and_create_csv()
#container.sort()
container = extract_data_from_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
pretrained_model = EffNet.load_from_checkpoint(checkpoint_path = '../input/fresh-epoch-0/epoch0-step3725.ckpt')
pretrained_model.eval()
pretrained_model.freeze()

my_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor()])


m = nn.Softmax()

if(len(container) > 559):
    for idx in range(len(container)):#
        #start_t = time.time()
        rgb,seg_mask,imageid = get_segmented_image_and_label(container,idx)
        #seg_t = time.time()
        #print("Seg: ",(seg_t - start_t))
        thresholded = convert_to_binary(seg_mask)
        contours, hierarchy = cv2.findContours(thresholded.astype(np.uint8).copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        predictionstring = []

        cv2.drawContours(thresholded, contours, -1, color=(0,0,0),thickness=-1)
    #     print(len(contours))
    #     print(len(np.unique(seg_mask)) - 1)
#         fig, ax = plt.subplots(1,2, figsize=(20,50))
        for i,c in enumerate(contours):
            rle = ""
            x, y, w, h = cv2.boundingRect(c)

            cropped_rgb = rgb[y:y+h,x:x+w]

#             ax[0].imshow(cropped_rgb)
#             ax[0].axis('off')
            cv2.drawContours(thresholded, contours, i, color=(255,255,255),thickness=-1)



#             ax[1].imshow(thresholded)
#             ax[1].axis('off')



            torch_img = torch.from_numpy(cropped_rgb)
            torch_img = torch_img.permute(2,1,0)

            transfromed_sample = my_transform(torch_img)

            rle = binary_mask_to_rle(thresholded)


            transfromed_sample = transfromed_sample.unsqueeze(0)
            y_hat = pretrained_model(transfromed_sample)
            conf = m(y_hat)
            y_hat_ = torch.argmax(y_hat,dim=1)
            #y_hat_top_4 = torch.topk(conf, 4)
            #print(y_hat)
            #print(y_hat_top_4)
            conf_hat = torch.argmax(conf,dim=1)
            x = conf[0][conf_hat].numpy().tolist()


            predictionstring.extend([" ".join([str(y_hat_[0].numpy()),str(x[0]),rle])])

        #looping = time.time()
        #print("Looping:",(looping - start_t))

        predictionstring = " ".join(predictionstring)
        sub.loc[idx,'PredictionString'] = predictionstring


sub.to_csv('./submission.csv',index=False)
